# Code for "Object-level Planning with Large Language Models" (ICRA 2025)

## Step 0 :- Initialization

### Import libraries for LLM prompting

In [52]:
from olp_lib import *

from IPython.display import Image

# NOTE: keep checking the pricing for details on best model to use: https://openai.com/api/pricing/

# -- use the custom-made class for accessing OpenAI models:
openai_driver = OpenAIInterfacer(
	model_embed="text-embedding-3-small",
    model_text_gen="gpt-4o"
)

# NOTE: all incontext examples will be stored within a JSON file:
# -- Q: can we randomly sample from the set of incontext examples?
# -- Q: should we also select an example "closest" to the provided task?

incontext_file = "all_fewshot_examples.json"
fewshot_examples = json.load(open(incontext_file))

In [103]:
loaded_foons = []
for foon_file in os.listdir('./foon_prototypes/'):
    fga._resetFOON(); fga._constructFOON(os.path.join('./foon_prototypes/', foon_file))
    loaded_foons.append({
        "foon": fga.FOON_lvl3,
        "summary": llm_summarize_FOON(openai_driver, fga.FOON_lvl3, verbose=True)
    })

print(loaded_foons[0]["summary"])

  -- Reading file line : 100%|██████████| 30/30 [00:00<00:00, 12289.20it/s]


 -- [FOON-fga] : Opening FOON file named './foon_prototypes/foon_prototype_1.txt'...
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
{
    "step": 1,
    "action": "pick and place",
    "required_objects": [
        "first block",
        "second block"
    ],
    "object_states": {
        "first block": {
            "preconditions": [
                "under nothing",
                "on table"
            ],
            "effects": [
                "under second block",
                "on table"
            ]
        },
        "second block": {
            "preconditions": [
                "under nothing",
                "on table"
            ],
            "effects": [
                "on first block",
                "under not


  -- Reading file line : 100%|██████████| 22/22 [00:00<?, ?it/s]

1. Pick and place the first block under the second block.
2. Pick and place the second block under the third block.

 -- [FOON-fga] : Opening FOON file named './foon_prototypes/foon_prototype_2.txt'...
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
{
    "step": 1,
    "action": "pick and place",
    "required_objects": [
        "first block",
        "second block",
        "third block"
    ],
    "object_states": {
        "first block": {
            "preconditions": [
                "under nothing",
                "on table"
            ],
            "effects": [
                "under second block",
                "on table"
            ]
        },
        "second block": {
            "preconditions": [
                "under nothing",
                "on table"
            ],
            "effects"

1. **Pick and Place Action:**
   - **Required Objects:** First block, second block, third block
   - **Object States:**
     - **First Block:**
       - Preconditions: Under nothing, on table
       - Effects: Under second block, on table
     - **Second Block:**
       - Preconditions: Under nothing, on table
       - Effects: On first block, under third block
     - **Third Block:**
       - Preconditions: Under nothing, on table
       - Effects: Under nothing, on second block
None


### Initialize simulation environment

In [91]:
from utamp.generate import *

import utamp.driver as driver

# NOTE: we will potentially have 3 task scenes:
#   1. 'blocks' :- a scene focusing on block stacking
#           (either based on colour or alphabet);
#   2. 'packing' :- a scene focusing on packing away "toys" in boxes;
#   3. 'cocktail' :- a scene focusing on the ability to make different
#           cocktails based on available ingredients
# setting = choice(['blocks', 'packing', 'cocktail'])
setting = choice(['blocks',])

run_simulation = True

is_alphabetic = True
add_tiles = False

if run_simulation:

    all_colours = [
        {'red': [204, 0, 0]},
        {'green': [0, 204, 0]},
        {'blue': [0, 0, 204]},
        {'yellow': [204, 204, 0]},
        {'pink': [204, 0, 204]},
        {'purple': [102, 0, 204]},
        {'orange': [255, 128, 0]},
        {'black': [64, 64, 64]},
        {'white': [250, 250, 250]},
    ]

    min_attributes = 2

    # -- we will randomize the scene of blocks with a random subset of colours:
    subset_colours = {}
    for C in sample(all_colours, randint(min_attributes, len(all_colours))):
        subset_colours.update(C)

    all_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']

    attributes = {
        'colours': subset_colours,
        'alphabets': {
            x: os.path.join(os.getcwd(), './utamp/textures/', f'{x.lower()}.png') for x in sample(all_letters, randint(min_attributes, len(all_letters)))
        }
    }

    # is_alphabetic = randint(0,1)
    # add_tiles = bool(randint(0,1))

    # -- we have different functions depending on the scene type:
    fpath, tally = eval((f"randomize_{setting}("
                         f"'./utamp/scenes/panda_{setting}_prototype.ttt',"
                         "attributes,"
                         f"block_type={is_alphabetic},"
                         f"add_tiles={add_tiles})"))

    object_phrases = []
    for item in tally:
        for C in tally[item]:
            if bool(tally[item][C]):
                object_phrases.append(
                        f"{tally[item][C]} "
                        # + str(('' if not bool(is_alphabetic) else '\'') + f"{C}" + ('' if not bool(is_alphabetic) else '\''))
                        f"{C}{(' ' if not bool(is_alphabetic) else '-')}{item}{'s' if tally[item][C] > 1 else ''}"
                )

    sim_interfacer = driver.Interfacer(scene_file_name=fpath)

    print(f"\n{'*' * 4} AFTER COPPELIASIM RANDOMIZATION: {'*' * 4}")
    print("objects presently in scene:", sim_interfacer.objects_in_sim)
    print("\nobjects in sim for LLM:", object_phrases)

    user_tasks = [
        {
            "objects": object_phrases,
            "query": None
        }
    ]


**** AFTER COPPELIASIM RANDOMIZATION: ****
objects presently in scene: ['K_block_1', 'K_block_2', 'A_block_1', 'A_block_2', 'A_block_3', 'I_block_1', 'I_block_2', 'B_block_1', 'B_block_2', 'B_block_3', 'C_block_1']

objects in sim for LLM: ['2 K-blocks', '3 A-blocks', '2 I-blocks', '3 B-blocks', '1 C-block']


In [92]:
if not run_simulation:
    # NOTE: these are some predefined object sets and tasks just to illustrate how the method works:
	user_tasks = [
        {
            "objects": ["3 green blocks", "2 blue blocks","1 red block"],
            "query": "How can I stack a tower of primary colored blocks?"
        },
        {
            "objects": ["shaker of black pepper", "shaker of salt", "can of tomato juice", "cup of lemon juice", "celery",
                        "knife", "drinking glass", "spoon",
                        "bottle of vodka", "bottle of olive oil", "bottle of Worcestershire sauce"],
            "query": "How can I make a Bloody Mary cocktail?"
        },
        {
            "objects": ["2 pink toys", "1 green toy", "1 green box"],
            "query": "How can I pack each box with toys of its colour?"
        }
    ]

### Select Planning/Execution Methodology

In [93]:
all_methods = [
    'OLP', 'OLP-UTAMP',
	'FOON-UTAMP', 'FOON',
    'LLM-Planner', 'LLM-Planner-UTAMP',
    'LLM-UTAMP',
    'LLM+P'
]

use_cache = False

evaluated_methods = 'FOON'
# evaluated_methods = ['OLP-UTAMP', 'OLP', 'LLM-Planner', 'LLM-Planner-UTAMP', 'LLM+P']

### Prompt User for Task:

In [82]:
user_task = choice(user_tasks)

while not user_task['query']:
    user_task['query'] = input(f"The robot has the following objects available to it on a table: {str(user_task['objects'])}.\nWhat would you like the robot to do?")

print('User Task:', user_task['query'])

task_file_name = f"{str.lower(re.compile('[^a-zA-Z]').sub('_', user_task['query'])[:-1]).replace('__', '_')}.txt"

User Task: Organize the table into stacks of similar blocks.


## Step 1 :- LLM -> Object-level Plan

1. Initialize libraries needed for FOON (``FOON_graph_analyser.py``) as well as OpenAI api.

2. Perform 2-stage prompting for recipe prototype.
    - In the first stage, we ask the LLM for a *high-level recipe* (list of instructions) and a *list of objects* needed for completing the recipe.
    - In the second stage, we ask the LLM for a breakdown of *state changes* that happen for each step of the recipe; specifically, we ask for the *preconditions* and *effects* of each action, which is similar to how a functional unit in FOON has *input* and *output object nodes*.

### LLM Prompting for OLP Information

In [104]:
print('User Task:', user_task['query'])
print('Available objects:', user_task['objects'])

if bool(set(['FOON', 'FOON-UTAMP']) & set(evaluated_methods)):

    llm_output = generate_from_FOON(
        openai_obj=openai_driver,
        query=user_task['query'],
        FOON_samples=loaded_foons,
        scenario={
            "name": setting,
            "objects": user_task['objects'] if isinstance(user_task, dict) else None,
        },
        system_prompt_file='llm_prompts/foon_system_prompt.txt',
        verbose=True
    )

    # -- save unparsed output to a JSON file:
    json.dump(llm_output, open('raw_OLP.json', 'w'), indent=4)

    print("\nTASK:",  user_task)
    print(f"\n{'*' * 25}\n    High level plan\n{'*' * 23}\n{llm_output['language_plan']}\n")
    print(f"\All Objects: {llm_output['all_objects']}")
    print(f"\n\n{'*' * 25}\n    DETAILED \n{'*' * 23}\n{json.dumps(llm_output['object_level_plan'], indent=4)}")


User Task: Organize the table into stacks of similar blocks.
Available objects: ['1 J-block', '3 G-blocks', '2 L-blocks', '3 B-blocks', '3 F-blocks', '3 I-blocks']
***************************************************************************
Model: gpt-4o
Complete prompt:
[
    {
        "role": "system",
        "content": "You are a helpful assistant that will generate plans for robots. You will be given the following:\n  1. A simple plan sketch, with which you will generate an entirely new plan sketch.\n  2. A list of objects available to the robot.\n\nNote the following rules:\n- Assume that all objects are available on the table or work surface in front of the robot.\n- If there are objects that are not relevant to the task prompt, omit them from the generated plan.\n- Use one action verb per step. However, any steps involving \"pick\" or \"place\" must be written as a single step with the verb \"pick and place\".\n- When describing an object's precondition or effect, use the states

BdbQuit: 

In [34]:
print('User Task:', user_task['query'])
print('Available objects:', user_task['objects'])

if bool(set(['OLP', 'OLP-UTAMP']) & set(evaluated_methods)):

    llm_output = None

    if use_cache:
        while True:
            llm_output = repair_olp(
                openai_obj=openai_driver,
                query=user_task['query'],
                available_objects=user_task['objects'],
                verbose=False
            )

            if isinstance(llm_output, int):
                if llm_output == 0:
                    # -- return 0 means that there are no items in cache:
                    llm_output = None
                    break
                elif llm_output == -1:
                    # -- return -1 means that some error was encountered (e.g. LLM did not say "okay"):
                    pass

            else: break

    if not llm_output:
        # -- stick to the regular OLP creation pipeline powered by LLM goodness:
        while True:
            try:
                llm_output, chat_history = generate_olp(
                    openai_driver,
                    query=user_task["query"],
                    fewshot_examples=fewshot_examples,
                    scenario={
                        "name": setting,
                        "objects": user_task['objects'] if isinstance(user_task, dict) else None,
                    },
                    stage1_sys_prompt_file="llm_prompts/olp_stage1_prompt.txt",
                    stage2_sys_prompt_file="llm_prompts/olp_stage2_prompt.txt",
                    stage3_sys_prompt_file="llm_prompts/olp_stage3_prompt.txt",
                    stage4_sys_prompt_file="llm_prompts/olp_stage4_prompt.txt",
                    verbose=False
                )
            except Exception as e: print('-- something went wrong:', type(e), e.args)
            else:
                if llm_output: break

    # -- save unparsed output to a JSON file:
    json.dump(llm_output, open('raw_OLP.json', 'w'), indent=4)

    print("\nTASK:",  user_task)
    print(f"\n{'*' * 25}\n    High level plan\n{'*' * 23}\n{llm_output['language_plan']}\n")
    print(f"\All Objects: {llm_output['all_objects']}")
    print(f"\n\n{'*' * 25}\n    DETAILED \n{'*' * 23}\n{json.dumps(llm_output['object_level_plan'], indent=4)}")


User Task: Organize the table by making stacks of similar blocks.
Available objects: ['3 C-blocks', '3 G-blocks', '1 E-block', '2 J-blocks']

TASK: {'objects': ['3 C-blocks', '3 G-blocks', '1 E-block', '2 J-blocks'], 'query': 'Organize the table by making stacks of similar blocks.'}

*************************
    High level plan
***********************
1. Pick up the first C-block and place it on the second C-block.
2. Pick up the third C-block and place it on the first C-block.
3. Pick up the first G-block and place it on the second G-block.
4. Pick up the third G-block and place it on the first G-block.
5. Pick up the first J-block and place it on the second J-block.

\All Objects: ['first c-block', 'second c-block', 'third c-block', 'first g-block', 'second g-block', 'third g-block', 'first j-block', 'second j-block']


*************************
    DETAILED 
***********************
{
    "task_prompt": "Organize the table by making stacks of similar blocks.",
    "all_objects": [
 

## Step 2 :- Object-level Plan -> FOON

### Creating new functional units

In [35]:
if bool(set(['OLP', 'OLP-UTAMP']) & set(evaluated_methods)):

    FOON_prototype = []

    for x in range(len(llm_output['object_level_plan']['all_instructions'])):

        new_unit = olp_to_FOON(llm_output, index=x)

        # NOTE: in order to define a macro-problem, we need to properly identify all goal nodes;
        #       we will do this with the help of the LLM:
        if 'termination_steps' in llm_output and (x+1) in llm_output['termination_steps']:
            # -- set output objects as goal nodes for the functional units deemed as terminal steps:
            print(f'\nFunctional unit {x+1} has terminal goals!')
            for N in range(new_unit.getNumberOfOutputs()):
                new_unit.getOutputNodes()[N].setAsGoal()

        # -- add the functional unit to the FOON prototype:

        if not new_unit.isEmpty():
            # -- we should only add a new functional unit if it is not empty, meaning it must have the following:
            #    1. >=1 input node and >= 1 output node
            #    2. a valid motion node
            FOON_prototype.append(new_unit)
            FOON_prototype[-1].print_functions[-1]()
        else:
            print('NOTE: the following functional unit has an error, so skipping it:')
            new_unit.print_functions[-1]()

        print('\n', '*' * 40)

        print()

Creating functional unit for Step 1: Pick up the first C-block and place it on the second C-block.
	-> related objects: ['first C-block', 'second C-block'] 

Current object is : first C-block
	 first C-block state changes: {'preconditions': ['on table', 'under nothing'], 'effects': ['on second C-block', 'under nothing']}
		 preconditions : on table || related objects: table
		 preconditions : under nothing || related objects: nothing
		 effects : on second C-block || related objects: second C-block
		 effects : under nothing || related objects: nothing
Current object is : second C-block
	 second C-block state changes: {'preconditions': ['on table', 'under nothing'], 'effects': ['on table', 'under first C-block']}
		 preconditions : on table || related objects: table
		 preconditions : under nothing || related objects: nothing
		 effects : on table || related objects: table
		 effects : under first C-block || related objects: first C-block

Functional unit 1 has terminal goals!
O	first 

### Save Generated FOON
1. Use ```FOON_parser.py``` to clean and ensure FOON labels are correct
2. Add the cleaned FOON to the embedded cache of object-level plans

In [36]:
if bool(set(['OLP', 'OLP-UTAMP']) & set(evaluated_methods)):

    temp_file_name = 'prototype.txt'

    verbose = False

    # -- save the prototype FOON graph as a text file, which we will then run with a parser to correct numbering:
    if not os.path.exists('./preprocess/'):
        os.makedirs('./preprocess/')

    if not os.path.exists('./postprocess/'):
        os.makedirs('./postprocess/')

    with open(f'./preprocess/{temp_file_name}', 'w') as prototype:
        prototype.write(f"//\tFOON Prototype\n")
        prototype.write(f"//\t-- Task Prompt: {user_task['query']}\n")
        prototype.write(f"//\t-- Required Objects: {llm_output['all_objects']}\n")
        prototype.write("//\n")
        for unit in FOON_prototype:
            prototype.write(unit.getFunctionalUnitText())
            if verbose:
                print(f'{unit.print_functions[2]()}//')

    # -- running parsing module to ensure that FOON labels and IDs are made consistent for further use:
    #		(it is important that each object and state type have a *UNIQUE* identifier)
    fpa.skip_JSON_conversion = True		# -- we don't need JSON versions of a FOON
    fpa.skip_index_check = True			# -- always create a new set of index files

    fpa.source_dir = './preprocess/'
    fpa.target_dir = './postprocess/'
    fpa._run_parser()

    with open(f'./postprocess/{temp_file_name}', 'r') as input_file:
        with open(f'./postprocess/{task_file_name}', 'w') as output_file:
            for line in input_file:
                output_file.write(line)

    if verbose:
        print(f"\n-- [LLM-to-OLP] : File has been saved as \"./postprocess/{task_file_name}\"")
        print()
        print(os.path.join(os.getcwd(), './postprocess/', task_file_name))

    fga._resetFOON()
    fga._constructFOON(f'./postprocess/{task_file_name}')

    embed_olp(
        openai_driver,
        llm_output,
        func_units=fga.FOON_functionalUnits[-1]
    )

    print(f"\n-- [LLM-to-OLP] : a new FOON has been added to embedding file!")

-- [FOON_parser] : Initiating parsing procedure!

 -- [FOON_parser] : Commencing parsing...
  -- parsing 'prototype.txt'...

 -- [FOON_parser] : Saving corrected files to './postprocess/'...
  -- Saving 'prototype.txt'...

-- Revising object label senses using WordNet and Concept-Net...



  -- Reading file line : 100%|██████████| 74/74 [00:00<00:00, 73410.24it/s]


SUMMARY OF CHANGES:
  -- new total of OBJECTS : 10
  -- new total of STATES : 2
  -- new total of MOTIONS : 1

 -- [FOON_parser] : Parsing complete!

 -- [FOON-fga] : Opening FOON file named './postprocess/organize_the_table_by_making_stacks_of_similar_blocks.txt'...
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to thi


-- [LLM-to-OLP] : a new FOON has been added to embedding file!


## Step 3 :- FOON -> ```FOON-TAMP```/```FOON_to_PDDL```
1. Parse FOON file -- this step is important to ensure that all labels are unique and that the generated file follows the FOON syntax.

2. Run ``FOON_to_PDDL.py`` script to generate FOON macro-operators

### Generate PDDL files using ```FOON_to_PDDL``` module

In [37]:
import FOON_TAMP as fta

use_macro_planning = False

all_macro_POs = None

if bool(set(['OLP', 'OLP-UTAMP']) & set(evaluated_methods)):

    FOON_subgraph_file = f'./postprocess/{task_file_name}'
    micro_problems_path = './micro_problems-' + Path(FOON_subgraph_file).stem
    robot_domain_file = './robot_skills.pddl'

    # -- definition of macro and micro plan file names:
    macro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_macro.plan'
    micro_plan_file = os.path.splitext(FOON_subgraph_file)[0] + '_micro.plan'

    fta.micro_domain_file = robot_domain_file

    fta.flag_perception = False

    # -- create a new folder for the generated problem files and their corresponding plans:
    fta.micro_problems_dir = micro_problems_path
    if not os.path.exists(fta.micro_problems_dir):
        os.makedirs(fta.micro_problems_dir)

    ftp = fta.ftp

    # -- perform conversion of the FOON subgraph file to PDDL:
    ftp.FOON_subgraph_file = FOON_subgraph_file
    ftp._convert_to_PDDL('OCP')

    ## -- parse through the newly created domain file and find all (macro) planning operators:
    all_macro_POs = {PO.name : PO for PO in fta._parseDomainPDDL(ftp.FOON_domain_file)}

  -- Reading file line : 100%|██████████| 74/74 [00:00<00:00, 32126.95it/s]


 -- [FOON-fga] : Opening FOON file named './postprocess/organize_the_table_by_making_stacks_of_similar_blocks.txt'...
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!
 -- WARNING: unchanged object node is being added to this functional unit!

 -- [FOON-fga] : Building internal dictionaries...
 -- [FOON-fga] : Building output-to-FU dictionaries...
  -- Level 1: Output obj

In [38]:
if use_macro_planning:

    # -- checking to see if a macro level plan has been found (FD allows exporting to a file):
    outcome = fta._findPlan(
        domain_file=ftp.FOON_domain_file,   # NOTE: FOON_to_PDDL object will already contain the names of the
        problem_file=ftp.FOON_problem_file,  #   corresponding macro-level domain and problem files
        output_plan_file=macro_plan_file
    )

    # TODO: write planning pipeline for other planners?

    complete_micro_plan = []

    if fta._checkPlannerOutput(output=outcome):
        print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was found!")

        # -- now that we have a plan, we are going to parse it for the steps:
        macro_file = open(macro_plan_file, 'r')

        # NOTE: counters for macro- and micro-level steps:
        macro_count, micro_count = 0, 0

        macro_plan_lines = list(macro_file)
        for L in macro_plan_lines:
            if L.startswith('('):
                print('\t\t\t' + str(macro_count) + ' : ' + L.strip())
        print()

        for L in range(len(macro_plan_lines)):

            if macro_plan_lines[L].startswith('('):
                # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
                #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
                #	2. effects - this will become a sub-problem file's goal states (as predicates)

                macro_count += 1

                macro_PO_name = macro_plan_lines[L][1:-2].strip()

                print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

                # -- try to find this step's matching planning operator definition:
                matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

                # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
                if matching_PO_obj:
                    # -- create sub-problem file (i.e., at the micro-level):
                    micro_problem_file = fta._defineMicroProblem(
                        macro_PO_name,
                        preconditions=matching_PO_obj.getPreconditions(),
                        effects=matching_PO_obj.getEffects(),
                    )

                    micro_domain_file = fta._defineMicroDomain(
                        preconditions=matching_PO_obj.getPreconditions(),
                    )

                    complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

                    need_to_replan = False

                    # -- try to find a sub-problem plan / solution:
                    outcome = fta._findPlan(
                        domain_file=micro_domain_file,
                        problem_file=micro_problem_file,
                        output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
                    )

                    print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

                    if fta._checkPlannerOutput(outcome):

                        print('\t\t -- micro-level plan found as follows:')

                        # -- open the micro problem file, read each line referring to a micro PO, and save to list:
                        micro_file = open(
                            str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

                        # -- all except for the last line should be valid steps:
                        micro_file_lines = []
                        count = 0

                        for micro_line in micro_file:
                            if micro_line.startswith('('):
                                # -- parse the line and remove trailing newline character:
                                micro_step = micro_line.strip()
                                micro_file_lines.append(micro_step)

                                # -- print entire plan to the command line in format of X.Y,
                                #       where X is the macro-step count and Y is the micro-step count:
                                count += 1
                                print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
                            #endif
                        #endfor
            print()
    else:
        print("  -- [FOON-TAMP] : A macro-level plan for '" + str(FOON_subgraph_file) + "' was NOT found!")


In [39]:
if use_macro_planning:

    complete_micro_plan = []

    # -- we can also perform a search where we assume we already have a task tree
    #		and we execute each functional unit one by one as we see

    macro_plan_lines = []
    for N in range(len(ftp.fga.FOON_lvl3)):
        PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'
        macro_plan_lines.append(PO_name)

    macro_count = 0

    for L in range(len(macro_plan_lines)):

        macro_count += 1

        macro_PO_name = macro_plan_lines[L]

        print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

        # -- try to find this step's matching planning operator definition:
        matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

        # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
        if matching_PO_obj:
            # -- create sub-problem file (i.e., at the micro-level):
            micro_problem_file = fta._defineMicroProblem(
                macro_PO_name,
                preconditions=matching_PO_obj.getPreconditions(),
                effects=matching_PO_obj.getEffects(),
            )

            micro_domain_file = fta._defineMicroDomain(
                preconditions=matching_PO_obj.getPreconditions(),
            )

            complete_micro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

            need_to_replan = False

            # -- try to find a sub-problem plan / solution:
            outcome = fta._findPlan(
                domain_file=micro_domain_file,
                problem_file=micro_problem_file,
                output_plan_file=str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan')
            )

            print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')

            if fta._checkPlannerOutput(outcome):

                print('\t\t -- micro-level plan found as follows:')

                # -- open the micro problem file, read each line referring to a micro PO, and save to list:
                micro_file = open(
                    str(fta.micro_problems_dir + '/' + macro_PO_name + '_micro.plan'), 'r')

                # -- all except for the last line should be valid steps:
                micro_file_lines = []
                count = 0

                for micro_line in micro_file:
                    if micro_line.startswith('('):
                        # -- parse the line and remove trailing newline character:
                        micro_step = micro_line.strip()
                        micro_file_lines.append(micro_step)

                        # -- print entire plan to the command line in format of X.Y,
                        #       where X is the macro-step count and Y is the micro-step count:
                        count += 1
                        print('\t\t\t' + str(macro_count) + '.' + str(count) + ' : ' + micro_step)
                    #endif
                #endfor
        print()

In [40]:
OMPL_algorithm = 'RRTConnect'
OMPL_max_compute = 5
OMPL_max_simplify = 5

## OLP (OMPL+skills)

In [42]:
# NOTE: all_methods = ['OLP', 'OLP-cache', 'OLP-UTAMP', 'OLP-cache-UTAMP', 'LLM-Planner', 'LLM-Planner-UTAMP', 'LLM-UTAMP', 'LLM+P']

import utamp.driver as driver

if bool(set(['OLP']) & set(evaluated_methods)):

    method = 'OLP'

    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()

    # NOTE: counters for macro- and micro-level steps:
    macro_count, micro_count = 0, 0

    macro_plan = []

    total_success = 0

    prev_obj_mapping = None

    for N in range(len(ftp.fga.FOON_lvl3)):
        # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
        #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
        #	2. effects - this will become a sub-problem file's goal states (as predicates)

        macro_count += 1

        macro_PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'

        print(" -- [FOON-TAMP] : Searching for micro-level plan for '" + macro_PO_name + "' macro-PO...")

        print(f"\n{'*' * 30}")
        ftp.fga.FOON_lvl3[N].print_functions[-1]()
        print(f"{'*' * 30}\n")

        # -- try to find this step's matching planning operator definition:
        matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

        # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
        if not matching_PO_obj:
            continue

        objects_in_OLP = []
        for O in ftp.fga.FOON_lvl3[N].getInputNodes() + ftp.fga.FOON_lvl3[N].getOutputNodes():
            objects_in_OLP.append(ftp._reviseObjectLabels(O.getObjectLabel()))
            for x in range(O.getNumberOfStates()):
                related_obj = O.getRelatedObject(x)
                if related_obj:
                    objects_in_OLP.append(ftp._reviseObjectLabels(related_obj))

        while True:
            # -- prompt LLMs to perform object grounding:
            grounding = llm_grounding_sim_objects(
                openai_driver,
                objects_in_OLP,
                sim_interfacer.objects_in_sim,
                {
                    'preconditions': matching_PO_obj.getPreconditions(),
                    'effects': matching_PO_obj.getEffects()
                },
                previous_mapping=prev_obj_mapping,
                verbose=False,
            )

            if grounding: break

        object_mapping, grounded_predicates = grounding[0], grounding[1]
        print(grounded_predicates)
        print(object_mapping)

        # -- keep track of what objects have already been grounded previously:
        previous_obj_mapping = object_mapping

        need_to_replan = False

        # -- create sub-problem file (micro-level/task-level):
        micro_problem_file = driver.create_problem_file(
            micro_fpath=micro_problems_path,
            action_name=macro_PO_name,
            preconditions=grounded_predicates['preconditions'],
            effects=grounded_predicates['effects'],
            state=sim_interfacer.perform_sensing(check_collision=False, verbose=False)
        )

        # -- create step-relevant domain file (micro-level/task-level):
        micro_domain_file = driver.create_domain_file(
            micro_fpath=micro_problems_path,
            template_domain_fpath=robot_domain_file,
            objects_in_sim=sim_interfacer.objects_in_sim,
            typing=llm_grounding_pddl_types(openai_driver, sim_interfacer.objects_in_sim),
        )

        print(micro_problem_file)
        print(micro_domain_file)

        print('\n\t' + 'step ' + str(macro_count) +' -- (' + macro_PO_name + ')')
        macro_plan.append('; step ' + str(macro_count) + ' -- (' + macro_PO_name + '):')

        # -- try to find a sub-problem plan / solution:
        if driver.solve(
            driver.find_plan(
                domain_file=micro_domain_file,
                problem_file=micro_problem_file,
                output_plan_file=str(micro_problems_path + '/' + macro_PO_name + '_micro.plan'),
                verbose=False,
            )
        ):
            print('\t-- micro-level plan found as follows:')

            # -- open the micro problem file, read each line referring to a micro PO, and save to list:
            micro_file = open(str(micro_problems_path + '/' + macro_PO_name + '_micro.plan'), 'r')

            # -- all except for the last line should be valid steps:
            micro_plan = []

            for micro_line in micro_file:
                if micro_line.startswith('('):
                    # -- parse the line and remove trailing newline character:
                    micro_step = micro_line.strip()
                    micro_plan.append(micro_step)

                    # -- print entire plan to the command line in format of X.Y,
                    #       where X is the macro-step count and Y is the micro-step count:
                    print('\t\t' + str(macro_count) + '.' + str(len(micro_plan)) + ' : ' + micro_step)

            micro_count += len(micro_plan)

            print(f"\n{'*' * 10} ROBOT EXECUTION {'*' * 10}")

            for x in range(len(micro_plan)):
                micro_step = micro_plan[x]
                print(f"-- running step {macro_count}.{x+1}: {micro_step} -- ", end="")

                if 'pick' in micro_step:
                    target_object = micro_step[1:-1].split(' ')[1]
                elif 'place' in micro_step:
                    target_object = micro_step[1:-1].split(' ')[2]

                # NOTE: seems like Fast-Downward makes everything lowercase...
                for obj in sim_interfacer.objects_in_sim:
                    if obj.lower() == target_object.lower():
                        target_object = obj

                print(target_object, f"pick={bool('pick' in micro_step)}", "...", end="")

                result = sim_interfacer.execute(
                    target_object,
                    gripper_action=int(bool('pick' in micro_step)),
                    algorithm=OMPL_algorithm,
                    max_compute=OMPL_max_compute,
                    max_simplify=OMPL_max_simplify,
                )

                print(f" {'success' if result else 'failed'}!")

                total_success += int(result)

        else:
            print('\t-- no micro-level plan found!')

        print()

    print(f"{'*' * 30}\n\n  RESULTS (method=\"{method}\"):")
    print(f"\t-- total number of macro-actions: {macro_count}")
    print(f"\t-- total number of micro-actions: {micro_count}")
    print(f"\t\t-- success: {total_success / micro_count * 100.0}% ({total_success} / {micro_count})")

    sim_interfacer.pause()

    time_taken = f'total time taken: {sim_interfacer.get_elapsed_time()}'
    print(f'\n{time_taken}')
    sim_interfacer.sim_print(time_taken)

    # -- take a screenshot of the final state of the world:
    final_state_img = os.path.join(os.getcwd(),f'{Path(task_file_name).stem}-{method}.png')
    sim_interfacer.take_snapshot(file_name=final_state_img)
    Image(filename=final_state_img)

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_0' macro-PO...

******************************
O	first c-block
S	under	 [nothing]
S1	on	 [table]
O3	second c-block
S	under	 [nothing]
S1	on	 [table]
M	pick and place	<Assumed>
O	first c-block
S	under	 [nothing]
S1	on	 [second c-block]
O3	second c-block
S	under	 [first c-block]
S1	on	 [table]
******************************



  -- Checking object properties in CoppeliaSim scene...:   0%|          | 0/9 [00:00<?, ?it/s]

{'preconditions': ['(on table C_block_1)', '(on C_block_1 air)', '(under C_block_2 table)', '(on C_block_2 air)', '(under C_block_1 table)', '(on table C_block_2)'], 'effects': ['(on C_block_2 C_block_1)', '(under C_block_1 C_block_2)', '(on C_block_1 air)', '(under C_block_2 table)', '(on table C_block_2)', '(not (on table C_block_1)', '(not (on C_block_2 air))', '(not (under C_block_1 table))']}
{'first_c_block': 'C_block_1', 'second_c_block': 'C_block_2'}


  -- Checking object properties in CoppeliaSim scene...: 100%|██████████| 9/9 [00:00<00:00, 66.50it/s]


./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\pick_and_place_0.pddl
./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\./robot_skills.pddl

	step 1 -- (pick_and_place_0)
	-- micro-level plan found as follows:
		1.1 : (pick g_block_3 c_block_1)
		1.2 : (place-on-top g_block_3 table)
		1.3 : (pick c_block_1 table)
		1.4 : (place-on-top c_block_1 c_block_2)

********** ROBOT EXECUTION **********
-- running step 1.1: (pick g_block_3 c_block_1) -- G_block_3 pick=True ...

...

./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\pick_and_place_2.pddl
./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\./robot_skills.pddl

	step 3 -- (pick_and_place_2)
	-- micro-level plan found as follows:
		3.1 : (pick g_block_1 table)
		3.2 : (place-on-top g_block_1 g_block_2)

********** ROBOT EXECUTION **********
-- running step 3.1: (pick g_block_1 table) -- G_block_1 pick=True ... success!
-- running step 3.2: (place-on-top g_block_1 g_block_2) -- G_block_2 pick=False ... success!

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_3' macro-PO...

******************************
O7	third g-block
S	under	 [nothing]
S1	on	 [table]
O5	first g-block
S	under	 [nothing]
S1	on	 [second g-block]
M	pick and place	<Assumed>
O7	third g-block
S1	on	 [first g-block]
S	under	 [nothing]
O5	first g-block
S1	on	 [second g-block]
S	under	 [third g-block]
******************************

{'preconditions': ['(on G_block_1 air)', '(on G_

  -- Checking object properties in CoppeliaSim scene...: 100%|██████████| 9/9 [00:00<00:00, 60.18it/s]


./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\pick_and_place_3.pddl
./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\./robot_skills.pddl

	step 4 -- (pick_and_place_3)
	-- micro-level plan found as follows:
		4.1 : (pick g_block_3 table)
		4.2 : (place-on-top g_block_3 g_block_1)

********** ROBOT EXECUTION **********
-- running step 4.1: (pick g_block_3 table) -- G_block_3 pick=True ... success!
-- running step 4.2: (place-on-top g_block_3 g_block_1) -- G_block_1 pick=False ... success!

 -- [FOON-TAMP] : Searching for micro-level plan for 'pick_and_place_4' macro-PO...

******************************
O8	first j-block
S	under	 [nothing]
S1	on	 [table]
O9	second j-block
S	under	 [nothing]
S1	on	 [table]
M	pick and place	<Assumed>
O8	first j-block
S	under	 [nothing]
S1	on	 [second j-block]
O9	second j-block
S	under	 [first j-block]
S1	on	 [table]
******************************

{'preconditions': ['(on J_block_2 air)', '(on table J_block_1)'

  -- Checking object properties in CoppeliaSim scene...: 100%|██████████| 9/9 [00:00<00:00, 39.36it/s]


./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\pick_and_place_4.pddl
./micro_problems-organize_the_table_by_making_stacks_of_similar_blocks\./robot_skills.pddl

	step 5 -- (pick_and_place_4)
	-- micro-level plan found as follows:
		5.1 : (pick j_block_1 table)
		5.2 : (place-on-top j_block_1 j_block_2)

********** ROBOT EXECUTION **********
-- running step 5.1: (pick j_block_1 table) -- J_block_1 pick=True ... success!
-- running step 5.2: (place-on-top j_block_1 j_block_2) -- J_block_2 pick=False ... success!

******************************

  RESULTS (method="OLP"):
	-- total number of macro-actions: 5
	-- total number of micro-actions: 12
		-- success: 100.0% (12 / 12)

-- total time taken: 203.59934163093567


## OLP (UTAMP)

**NOTE:** More on UTAMP [here](https://github.com/alejandroagostini/utamp).

In [43]:
# NOTE: all_methods = ['OLP', 'OLP-cache', 'OLP-UTAMP', 'OLP-cache-UTAMP', 'LLM-Planner', 'LLM-Planner-UTAMP', 'LLM-UTAMP', 'LLM+P']

import utamp.driver as driver

if bool(set(['OLP-UTAMP']) & set(evaluated_methods)):

    method = 'OLP-UTAMP'

    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()
    utamp_client = driver.UTAMP(config_fpath='./utamp_config.json')

    # NOTE: counters for macro-level steps (since we are using UTAMP, we do not need to do micro-planning):
    macro_count = 0
    total_success = 0

    previous_obj_mapping = None

    for N in range(len(ftp.fga.FOON_lvl3)):
        # NOTE: this is where we have identified a macro plan's step; here, we check the contents of its PO definition for:
        #	1. preconditions - this will become a sub-problem file's initial states (as predicates)
        #	2. effects - this will become a sub-problem file's goal states (as predicates)

        macro_count += 1

        macro_PO_name = f'{ftp._reviseObjectLabels(ftp.fga.FOON_lvl3[N].getMotion().getMotionLabel())}_{N}'

        print(" -- [FOON-TAMP] : Using UTAMP to solve '" + macro_PO_name + "' macro-PO...")

        # print(f"\n{'*' * 30}\n{ftp.fga.FOON_lvl3[N].print_functions[-1]()}\n{'*' * 30}\n")

        # -- try to find this step's matching planning operator definition:
        matching_PO_obj = all_macro_POs[macro_PO_name] if macro_PO_name in all_macro_POs else None

        # -- when we find the equivalent planning operator, then we proceed to treat it as its own problem:
        if not matching_PO_obj:
            continue

        objects_in_OLP = []
        for O in ftp.fga.FOON_lvl3[N].getInputNodes() + ftp.fga.FOON_lvl3[N].getOutputNodes():
            objects_in_OLP.append(ftp._reviseObjectLabels(O.getObjectLabel()))
            for x in range(O.getNumberOfStates()):
                related_obj = O.getRelatedObject(x)
                if related_obj:
                    objects_in_OLP.append(ftp._reviseObjectLabels(related_obj))

        while True:
            # -- prompt LLMs to perform object grounding:
            grounding = llm_grounding_sim_objects(
                openai_driver,
                objects_in_OLP=objects_in_OLP,
                objects_in_sim=sim_interfacer.objects_in_sim,
                original_predicates={
                    'preconditions': matching_PO_obj.getPreconditions(),
                    'effects': matching_PO_obj.getEffects()
                },
                previous_mapping=previous_obj_mapping,
                verbose=False
            )

            if grounding: break

        object_mapping, grounded_predicates = grounding[0], grounding[1]
        print(grounded_predicates)
        print(object_mapping)

        # -- keep track of what objects have already been grounded previously:
        previous_obj_mapping = object_mapping

        assert bool(grounded_predicates['effects']), f"Error: empty list of goals! Check the generated micro-problem file '{micro_problem_file}'"

        # NOTE: UTAMP does not process "not" relations; filter out the strings that refer to negated predicates:
        utamp_goals = [x for x in grounded_predicates['effects'] if not x.startswith('(not')]

        total_success += int(utamp_client.plan_and_execute(
            sim_interfacer,
            goals_for_utamp=utamp_goals,
            verbose=False,
            algorithm=OMPL_algorithm,
            max_compute=OMPL_max_compute,
            max_simplify=OMPL_max_simplify,
        ))

    print(f"{'*' * 30}\n\n  RESULTS (method=\"{method}\"):")
    print(f"\t-- total number of macro-actions: {macro_count}")
    print(f"\t\t-- success: {total_success / macro_count * 100.0}%")

    sim_interfacer.pause()

    time_taken = f'total time taken: {sim_interfacer.get_elapsed_time()}'
    print(f'\n{time_taken}')
    sim_interfacer.sim_print(time_taken)

    # -- take a screenshot of the final state of the world:
    final_state_img = os.path.join(os.getcwd(),f'{Path(task_file_name).stem}-{method}.png')
    sim_interfacer.take_snapshot(file_name=final_state_img)
    Image(filename=final_state_img)

 -- [FOON-TAMP] : Using UTAMP to solve 'pick_and_place_0' macro-PO...
{'preconditions': ['(on table C_block_1)', '(on C_block_1 air)', '(under C_block_2 table)', '(on C_block_2 air)', '(under C_block_1 table)', '(on table C_block_2)'], 'effects': ['(on C_block_2 C_block_1)', '(under C_block_1 C_block_2)', '(on C_block_1 air)', '(under C_block_2 table)', '(on table C_block_2)', '(not (on table C_block_1)', '(not (on C_block_2 air))', '(not (under C_block_1 table))']}
{'first_c_block': 'C_block_1', 'second_c_block': 'C_block_2'}
goals sent to utamp:	C_block_2:on,C_block_1;C_block_1:under,C_block_2:on,air;constraints:placefromabove,1;

********** UTAMP CLIENT/SERVER INTERACTION **********

status 0:	Waiting for goal from client.
status 10:	Waiting for goal from client.
status 0:	waiting for robot sensing
status 0:	sensing completed
status 10:	sensing completed
status 0:	waiting for robot sensing
status 0:	sensing completed
status 0:	Generating new plan.
status 10:	Generating new plan.
sta

### UTAMP/OMPL Tester

In [50]:
test = False

if test:
    utamp_client = driver.UTAMP(config_fpath='./utamp_config.json')
    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()

    result = utamp_client.plan_and_execute(
        sim_interfacer,
        goals_for_utamp=["(on G_block_1 G_block_2)", "(under G_block_2 air)"],
        pick_from_top=False,
        verbose=False)

    sim_interfacer.pause()


********** UTAMP CLIENT/SERVER INTERACTION **********

 -- goals sent to utamp:	G_block_1:on,G_block_2;G_block_2:under,air:on,air;

UTAMP interactions:
status 10:	Waiting for goal from client.
status 0:	waiting for robot sensing
status 0:	sensing completed
status 10:	sensing completed
status 0:	waiting for robot sensing
status 0:	sensing completed
status 0:	Generating new plan.
status 10:	Generating new plan.
status 0:	sensing completed
status 10:	sensing completed
status 0:	waiting for robot sensing
status 1:	sensing completed
status 10:	waiting for robot execution
status 0:	waiting for robot sensing
status 1:	waiting for robot execution
status 0:	waiting for robot execution
status 10:	waiting for robot execution
status 0:	waiting for robot sensing
status 0:	sensing completed
status 10:	sensing completed
status 0:	waiting for robot sensing
status 1:	waiting for robot execution
status 0:	waiting for robot execution
status 10:	waiting for robot execution
status 0:	waiting for robot sen

KeyboardInterrupt: 

In [45]:
test = False

if test:
    utamp_client = driver.UTAMP(config_fpath='./utamp_config.json')
    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()

    micro_step, target_object = 'pick', 'F_block_1'
    result = sim_interfacer.execute(
        target_object,
        gripper_action=int(bool('pick' in micro_step)),
        algorithm="PRMstar")

    micro_step, target_object = 'place', 'E_block_2'
    result = sim_interfacer.execute(
        target_object,
        gripper_action=int(bool('pick' in micro_step)),
        algorithm="PRMstar")


    micro_step, target_object = 'pick', 'A_block_1'
    result = sim_interfacer.execute(
        target_object,
        gripper_action=int(bool('pick' in micro_step)),
        algorithm="PRMstar")

    micro_step, target_object = 'place', 'F_block_1'
    result = sim_interfacer.execute(
        target_object,
        gripper_action=int(bool('pick' in micro_step)),
        algorithm="PRMstar")

## Step 4 :- Baselines

### LLM-Planner (OMPL+skills)

In [46]:
# NOTE: all_methods = ['OLP', 'OLP-cache', 'OLP-UTAMP', 'OLP-cache-UTAMP', 'LLM-Planner', 'LLM-Planner-UTAMP', 'LLM-UTAMP', 'LLM+P']

import utamp.driver as driver

user_task = {'query': "organize the table by making piles of similar blocks"}

if bool(set(['LLM-Planner']) & set(evaluated_methods)):

    method = 'LLM-Planner'

    # -- use the LLM as a planner to acquire a task plan, where each step will be executed using programmed skills:
    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()

    # -- we are going to feed the LLM with the prompt of coming up with a plan using the pre-defined skills:
    llm_planner_sys_prompt = open('llm_prompts/llm_planner_system_prompt.txt', 'r').read()

    prompt_context = f"There is a scenario with the following objects: {sim_interfacer.objects_in_sim}. Please await further instructions."
    print("objects presently in scene:", sim_interfacer.objects_in_sim)

    interaction = [
        {"role": "system", "content": llm_planner_sys_prompt},
        {"role": "user", "content": prompt_context}
    ]

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    prompt_goal = (
        f"Your task is as follows: {user_task['query']}."
        " Transform this instruction into a PDDL goal specification in terms of 'on' relations. Do not add any explanation."
    )

    interaction.extend([{"role": "user", "content": prompt_goal}])

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    print(f"LLM-generated goal: {response}")

    llm_planner_user_prompt = (
        "Find a task plan in PDDL to achieve this goal given the initial state below."
        " Only specify the list of actions needed."
        " Use the actions defined above. Do not add any explanation.\n\n"
        f"Initial state:\n{sim_interfacer.perform_sensing(method=3).replace('air', 'nothing')}"
    )

    interaction.extend([{"role": "user", "content": llm_planner_user_prompt}])

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    steps = response.split('\n')
    parsed_steps = []

    # -- first, we need to parse through the plan that the LLM gives us by splitting:
    total_steps = 0

    while True:
        total_steps += 1
        found = False

        for x in range(len(steps)):
            if steps[x].startswith(f"{total_steps}."):
                parsed_steps.append(steps[x])
                found = True

        if not found: break

    print("Complete list of actions:")
    print("\n".join(parsed_steps))

    print(f"\n{'*' * 25}\n")

    # -- now that we know the total number of steps, we can go ahead and start to execute each step:
    total_success = 0

    print('LLM-Planner execution:')

    if parsed_steps:

        for x in range(len(parsed_steps)):
            # -- use regex to help us split the string into smaller components:
            micro_step = re.search("\(.+\)", parsed_steps[x])[0]

            if 'pick' in micro_step:
                target_object = micro_step[1:-1].split(' ')[1]
            elif 'place' in micro_step:
                target_object = micro_step[1:-1].split(' ')[2]

            print(f"-- running step {x+1}.: {micro_step} -- ", end="")

            # -- we will keep track of every successful execution:
            result = sim_interfacer.execute(
                target_object,
                gripper_action=int('pick' in micro_step),
                algorithm=OMPL_algorithm,
                max_compute=OMPL_max_compute,
                max_simplify=OMPL_max_simplify
            )

            print(f" {'success' if result else 'failed'}!")

            total_success += int(result)

        print('\n\n% Success rate:', float(total_success / len(parsed_steps) * 100.0), f"({total_success}/{len(parsed_steps)})" )

    sim_interfacer.pause()

    time_taken = f'total time taken: {sim_interfacer.get_elapsed_time()}'
    print(f'\n{time_taken}')
    sim_interfacer.sim_print(time_taken)

    # -- take a screenshot of the final state of the world:
    final_state_img = os.path.join(os.getcwd(),f'{Path(task_file_name).stem}-{method}.png')
    sim_interfacer.take_snapshot(file_name=final_state_img)
    Image(filename=final_state_img)

objects presently in scene: ['C_block_1', 'C_block_2', 'C_block_3', 'G_block_1', 'G_block_2', 'G_block_3', 'E_block_1', 'J_block_1', 'J_block_2']


  -- Checking object properties in CoppeliaSim scene...:   0%|          | 0/9 [00:00<?, ?it/s]

LLM-generated goal: Goal:
```
(on C_block_2 C_block_1)
(on C_block_3 C_block_2)
(on G_block_2 G_block_1)
(on G_block_3 G_block_2)
(on J_block_2 J_block_1)
```


  -- Checking object properties in CoppeliaSim scene...: 100%|██████████| 9/9 [00:00<00:00, 67.27it/s]


Complete list of actions:
1. (pick G_block_3 C_block_1)
2. (place G_block_3 table)
3. (pick C_block_2 table)
4. (place C_block_2 C_block_1)
5. (pick C_block_3 table)
6. (place C_block_3 C_block_2)
7. (pick G_block_2 table)
8. (place G_block_2 G_block_1)
9. (pick G_block_3 table)
10. (place G_block_3 G_block_2)
11. (pick J_block_2 table)
12. (place J_block_2 J_block_1)

*************************

LLM-Planner execution:
-- running step 1.: (pick G_block_3 C_block_1) -- 

  -- Checking object properties in CoppeliaSim scene...: 100%|██████████| 9/9 [00:00<00:00, 74.08it/s]

 success!
-- running step 2.: (place G_block_3 table) -- 

table grounding: found empty spot: /Dummy[3]
 success!
-- running step 3.: (pick C_block_2 table) --  success!
-- running step 4.: (place C_block_2 C_block_1) --  success!
-- running step 5.: (pick C_block_3 table) --  success!
-- running step 6.: (place C_block_3 C_block_2) --  success!
-- running step 7.: (pick G_block_2 table) --  success!
-- running step 8.: (place G_block_2 G_block_1) --  success!
-- running step 9.: (pick G_block_3 table) --  success!
-- running step 10.: (place G_block_3 G_block_2) --  success!
-- running step 11.: (pick J_block_2 table) --  success!
-- running step 12.: (place J_block_2 J_block_1) --  success!


% Success rate: 750.0 (90/12)

total time taken: 232.14750719070435


### LLM-Planner (UTAMP)

In [47]:
# NOTE: all_methods = ['OLP', 'OLP-cache', 'OLP-UTAMP', 'OLP-cache-UTAMP', 'LLM-Planner', 'LLM-Planner-UTAMP', 'LLM-UTAMP', 'LLM+P']

if bool(set(['LLM-Planner-UTAMP']) & set(evaluated_methods)):

    method = 'LLM-Planner-UTAMP'

    # -- use the LLM as a planner to acquire a task plan, where each step will be executed using programmed skills:
    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()
    utamp_client = driver.UTAMP(config_fpath='./utamp_config.json')

    # -- we are going to feed the LLM with the prompt of coming up with a plan using the pre-defined skills:
    llm_planner_sys_prompt = open('llm_prompts/llm_planner_system_prompt.txt', 'r').read()

    prompt_context = f"There is a scenario with the following objects: {sim_interfacer.objects_in_sim}. Please await further instructions."
    print("objects presently in scene:", sim_interfacer.objects_in_sim)

    interaction = [{"role": "system", "content": llm_planner_sys_prompt},
               {"role": "user", "content": prompt_context}]

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    prompt_goal = (
        f"Your task is as follows: {user_task['query']}."
        " Transform this instruction into a PDDL goal specification in terms of 'on' relations. Do not add any explanation."
    )

    interaction.extend([{"role": "user", "content": prompt_goal}])

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    print(f"LLM-generated goal: {response}")

    llm_planner_user_prompt = (
        "Find a task plan in PDDL to achieve this goal given the initial state below."
        " Only specify the list of actions needed."
        " Use the actions defined above. Do not add any explanation.\n\n"
        f"Initial state:\n{sim_interfacer.perform_sensing(method=3).replace('air', 'nothing')}"
    )

    interaction.extend([{"role": "user", "content": llm_planner_user_prompt}])

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    steps = response.split('\n')
    parsed_steps = []

    # -- first, we need to parse through the plan that the LLM gives us by splitting:
    total_steps = 0

    while True:
        total_steps += 1
        found = False

        for x in range(len(steps)):
            if steps[x].startswith(f"{total_steps}."):
                parsed_steps.append(steps[x])
                found = True

        if not found: break

    # -- now that we know the total number of steps, we can go ahead and start to execute each step:
    total_success = 0

    goals_per_step = []

    if parsed_steps:

        for x in range(len(parsed_steps)):
            # -- use regex to help us split the string into smaller components:
            micro_step = re.search("\(.+\)", parsed_steps[x])[0]

            # NOTE: since we are using UTAMP to simplify the task planning process,
            #       we only need to note the arguments for "place" actions.
            #   - UTAMP will reason about the necessary pick actions.
            if 'place' in micro_step:

                args = micro_step[1:-1].split(' ')

                goals_per_step.append([
                    f"(on {args[1]} air)",
                    f"(under {args[2]} {args[1]})",
                    f"(on {args[1]} {args[1]})",
                ])

                total_success += int(utamp_client.plan_and_execute(
                    sim_interfacer,
                    goals_for_utamp=goals_per_step[-1],
                    algorithm=OMPL_algorithm,
                    max_compute=OMPL_max_compute,
                    max_simplify=OMPL_max_simplify,
                ))

        print('\n\n% Success rate:', float(total_success / len(parsed_steps) * 100.0), f"({total_success}/{len(parsed_steps)})" )

    sim_interfacer.pause()

    time_taken = f'total time taken: {sim_interfacer.get_elapsed_time()}'
    print(f'\n{time_taken}')
    sim_interfacer.sim_print(time_taken)

    # -- take a screenshot of the final state of the world:
    final_state_img = os.path.join(os.getcwd(),f'{Path(task_file_name).stem}-{method}.png')
    sim_interfacer.take_snapshot(file_name=final_state_img)
    Image(filename=final_state_img)

objects presently in scene: ['C_block_1', 'C_block_2', 'C_block_3', 'G_block_1', 'G_block_2', 'G_block_3', 'E_block_1', 'J_block_1', 'J_block_2']
LLM-generated goal: Goal: 
```
(on C_block_1 C_block_2)
(on C_block_2 C_block_3)
(on G_block_1 G_block_2)
(on G_block_2 G_block_3)
(on J_block_1 J_block_2)
```


  -- Checking object properties in CoppeliaSim scene...: 100%|██████████| 9/9 [00:00<00:00, 40.61it/s]


goals sent to utamp:	G_block_3:on,air:on,G_block_3;constraints:placefromabove,1;

********** UTAMP CLIENT/SERVER INTERACTION **********

status 10:	waiting for robot sensing
status 0:	waiting for robot sensing
status 0:	sensing completed
status 10:	sensing completed
status 0:	waiting for robot sensing
status 0:	sensing completed
status -1:	Waiting for goal from client.
goals sent to utamp:	C_block_1:on,air:on,C_block_1;C_block_2:under,C_block_1:on,air;constraints:placefromabove,1;

********** UTAMP CLIENT/SERVER INTERACTION **********

status 10:	Waiting for goal from client.
status 0:	waiting for robot sensing
status 0:	sensing completed
status 10:	sensing completed
status 0:	waiting for robot sensing
status 0:	sensing completed
status 0:	Generating new plan.
status 0:	Failed plan generation.
status -3:	Waiting for goal from client.
goals sent to utamp:	C_block_2:on,air:on,C_block_2;C_block_3:under,C_block_2:on,air;constraints:placefromabove,1;

********** UTAMP CLIENT/SERVER INTERACT

KeyboardInterrupt: 

### LLM-for-UTAMP (Goal Generator)

In [ ]:
# NOTE: all_methods = ['OLP', 'OLP-cache', 'OLP-UTAMP', 'OLP-cache-UTAMP', 'LLM-Planner', 'LLM-Planner-UTAMP', 'LLM-UTAMP', 'LLM+P']

if bool(set(['LLM-Planner-UTAM']) & set(evaluated_methods)):
    method = 'LLM-Planner-UTAMP'

    # -- use the LLM as a planner to acquire a task plan, where each step will be executed using programmed skills:
    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()
    utamp_client = driver.UTAMP(config_fpath='./utamp_config.json')

    # -- we are going to feed the LLM with the prompt of coming up with a plan using the pre-defined skills:
    llm_planner_sys_prompt = open('llm_prompts/llm_planner_system_prompt.txt', 'r').read()

    prompt_context = f"There is a scenario with the following objects: {sim_interfacer.objects_in_sim}. Please await further instructions."
    print("objects presently in scene:", sim_interfacer.objects_in_sim)

    interaction = [{"role": "system", "content": llm_planner_sys_prompt},
               {"role": "user", "content": prompt_context}]

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    prompt_goal = (
        f"Your task is as follows: {user_task['query']}."
        "Transform this instruction into a PDDL goal specification in terms of 'on' relations. Do not add any explanation."
    )

    interaction.extend([{"role": "user", "content": prompt_goal}])

    _, response = openai_driver.prompt(interaction)
    interaction.extend([{"role": "assistant", "content": response}])

    steps = response.split('\n')
    parsed_steps = []

    # -- first, we need to parse through the plan that the LLM gives us by splitting:
    total_steps = 0

    while True:
        total_steps += 1
        found = False

        for x in range(len(steps)):
            if steps[x].startswith(f"{total_steps}."):
                parsed_steps.append(steps[x])
                found = True

        if not found: break

    # -- now that we know the total number of steps, we can go ahead and start to execute each step:
    total_success = 0

    goals_per_step = []

    for x in range(len(parsed_steps)):
        # -- use regex to help us split the string into smaller components:
        micro_step = re.search("\(.+\)", parsed_steps[x])[0]


        # NOTE: since we are using UTAMP to simplify the task planning process,
        #       we only need to note the arguments for "place" actions.
        #   - UTAMP will reason about the necessary pick actions.
        if 'place' in micro_step:

            args = micro_step[1:-1].split(' ')

            goals_per_step.append([
                f"(on {args[1]} air)",
                f"(under {args[2]} {args[1]})",
                f"(on {args[1]} {args[1]})",
            ])

            input(goals_per_step[-1])

            total_success += int(utamp_client.plan_and_execute(
                sim_interfacer,
                goal_preds=goals_per_step[-1],
                algorithm=OMPL_algorithm,
                max_compute=OMPL_max_compute,
                max_simplify=OMPL_max_simplify,
            ))

    print('\n\n% Successful:', total_success / (len(parsed_steps)) * 100.0)

    time_taken = sim_interfacer.get_elapsed_time()
    print(f'\n-- total time taken: {time_taken}')
    sim_interfacer.sim_print(time_taken)

    # -- take a screenshot of the final state of the world:
    final_state_img = os.path.join(os.getcwd(),f'{Path(task_file_name).stem}-{method}.png')
    sim_interfacer.take_snapshot(file_name=final_state_img)
    Image(filename=final_state_img)


### ~LLM+P (generate PDDL, run OMPL+skills)
 
**NOTE**: This method is akin to [LLM+P (Liu et al. 2023)](https://github.com/Cranial-XIX/llm-pddl/tree/main). We adopt a similar approach where we do the following steps:
1. Provide the LLM with an incontext example of a PDDL file for a similar task.
2. Provide context about the current state of the robot's environment (what objects and what relations are true).
3. Generate a problem file that will be compatible with a domain file.
4. Use a PDDL solver to find a task plan, and then run it with OMPL-based skills.

In [ ]:
# NOTE: all_methods = ['OLP', 'OLP-cache', 'OLP-UTAMP', 'OLP-cache-UTAMP', 'LLM-Planner', 'LLM-Planner-UTAMP', 'LLM-UTAMP', 'LLM+P']

if bool(set(['LLM+P']) & set(evaluated_methods)):

    method = 'LLM+P'

    sim_interfacer = driver.Interfacer(scene_file_name=fpath); sim_interfacer.start()

    llm_plus_p_dir = 'output_llm+p'
    if not os.path.exists(llm_plus_p_dir):
        os.mkdir(llm_plus_p_dir)

    pddl_sys_prompt_file = "llm_prompts/llm+p_system_prompt.txt"

    pddl_system_prompt = open(pddl_sys_prompt_file, 'r').read().replace('<problem_file_example>', top_fewshot_examples(
        openai_driver, fewshot_examples, user_task['query'], method=['llm+p', setting],)[0]['pddl'])

    message = [{"role": "system", "content": pddl_system_prompt}]

    pddl_user_prompt = ("Now I have a new planning problem and its description is as follows:\n"
                        f"These objects are on the table: {sim_interfacer.objects_in_sim}."
                        f" The current state of the world is:\n{sim_interfacer.perform_sensing(method=3, check_collision=False)}.")

    message.extend([{"role": "user", "content": pddl_user_prompt}])

    pddl_user_prompt = (f"\nYour goal is to achieve this task: {user_task['query']}."
                        "Provide me with the problem PDDL file that describes the new planning problem directly without further explanations.")

    message.extend([{"role": "user", "content": pddl_user_prompt}])

    _, response = openai_driver.prompt(chat_history=message, verbose=False)

    message.extend([{"role": "assistant", "content": response}])
    print(json.dumps(message, indent=4))

    llm_plus_p_pddl_files = f"{llm_plus_p_dir}/problem-{Path(task_file_name).stem}"

    if not os.path.exists(llm_plus_p_pddl_files):
        os.mkdir(llm_plus_p_pddl_files)

    llm_plus_p_problem_file = f"{llm_plus_p_pddl_files}/problem.pddl"

    with open(llm_plus_p_problem_file, "w") as llm_plus_p_problem:
        llm_plus_p_problem.write(parse_llm_code(response, separator="\n"))

    # -- create step-relevant domain file (task-level):
    micro_domain_file = driver.create_domain_file(
        micro_fpath=llm_plus_p_pddl_files,
        template_domain_fpath=robot_domain_file,
        objects_in_sim=sim_interfacer.objects_in_sim,
        domain_name=setting,
        typing=llm_grounding_pddl_types(openai_driver, sim_interfacer.objects_in_sim),
    )

    # -- try to find a sub-problem plan / solution:
    if driver.solve(
        driver.find_plan(
            domain_file=micro_domain_file,
            problem_file=llm_plus_p_problem_file,
            output_plan_file=f"{llm_plus_p_pddl_files}/final.plan",
            verbose=False,
        )
    ):
        print('\t-- micro-level plan found as follows:')

        # -- open the micro problem file, read each line referring to a micro PO, and save to list:
        micro_file = open(str(micro_problems_path + '/' + macro_PO_name + '_micro.plan'), 'r')

        # -- all except for the last line should be valid steps:
        micro_plan = []

        for micro_line in micro_file:
            if micro_line.startswith('('):
                # -- parse the line and remove trailing newline character:
                micro_step = micro_line.strip()
                micro_plan.append(micro_step)

                # -- print entire plan to the command line in format of X.Y,
                #       where X is the macro-step count and Y is the micro-step count:
                print('\t\t' + str(macro_count) + '.' + str(len(micro_plan)) + ' : ' + micro_step)

        print(f"\n{'*' * 10} ROBOT EXECUTION {'*' * 10}")

        for x in range(len(micro_plan)):
            micro_step = micro_plan[x]
            print(f"-- running step {macro_count}.{x+1}: {micro_step}... ", end="")

            if 'pick' in micro_step:
                target_object = micro_step[1:-1].split(' ')[1]
            elif 'place' in micro_step:
                target_object = micro_step[1:-1].split(' ')[2]

            # NOTE: seems like Fast-Downward makes everything lowercase...
            for obj in sim_interfacer.objects_in_sim:
                if obj.lower() == target_object.lower():
                    target_object = obj

            print(target_object, 'pick=', bool('pick' in micro_step), '...', end='')

            result = sim_interfacer.path_planning(
                target_object,
                gripper_action=int('pick' in micro_step),
                algorithm=OMPL_algorithm,
                max_compute=OMPL_max_compute,
                max_simplify=OMPL_max_simplify,
            )

            print(f" {'success' if result else 'failed'}!")

            total_success += int(result)

        print(f"{'*' * 30}\n\n  RESULTS (method=\"{method}\"):")
        print(f"\t-- total number of micro-actions: {len(micro_plan)}")
        print(f"\t\t-- success: {total_success / len(micro_plan) * 100.0}%")


    else:
        print(f"{'*' * 30}\n\n  RESULTS (method=\"{method}\"):")
        print("\t-- no plan found!\n\t\tsuccess: 0%")

    sim_interfacer.pause()

    time_taken = sim_interfacer.get_elapsed_time()
    print(f'\n-- total time taken: {time_taken}')
    sim_interfacer.sim_print(time_taken)

    # -- take a screenshot of the final state of the world:
    final_state_img = os.path.join(os.getcwd(),f'{Path(task_file_name).stem}-{method}.png')
    sim_interfacer.take_snapshot(file_name=final_state_img)
    Image(filename=final_state_img)